In [1]:
from torch import nn
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pytorch_lightning as pl
from pytorch_lightning.core.decorators import auto_move_data
from torchvision.models import resnet18, googlenet
from pytorch_lightning.metrics.functional import accuracy

from dlc_practical_prologue import generate_pair_sets
from models import *

In [2]:
# Load data
train_input, train_target, train_classes, test_input, test_target, test_classes = generate_pair_sets(nb=1000)

# Prepare dataloader with class variable (Used to test auxiliary networks)
ds_train_class = torch.utils.data.TensorDataset(train_input[:, 0:1, :, :], train_classes[:, 0])
ds_test_class = torch.utils.data.TensorDataset(test_input[:, 0:1, :, :], test_classes[:, 0])
dl_train_class = torch.utils.data.DataLoader(ds_train_class, batch_size=32, shuffle=True, num_workers=4)
dl_test_class = torch.utils.data.DataLoader(ds_test_class, batch_size=32, shuffle=False, num_workers=4)

# Prepare dataloader with target variable (Used for networks without auxiliary loss)
ds_train_target = torch.utils.data.TensorDataset(train_input, train_target)
ds_test_target = torch.utils.data.TensorDataset(test_input, test_target)
dl_train_target = torch.utils.data.DataLoader(ds_train_target, batch_size=32, shuffle=True, num_workers=4)
dl_test_target = torch.utils.data.DataLoader(ds_test_target, batch_size=32, shuffle=False, num_workers=4)

# Prepare dataloader with target and class variable (Used for networks with auxiliary loss)
ds_train_all = torch.utils.data.TensorDataset(train_input, train_classes, train_target)
ds_test_all = torch.utils.data.TensorDataset(test_input, test_classes, test_target)
dl_train_all = torch.utils.data.DataLoader(ds_train_all, batch_size=32, shuffle=True, num_workers=4)
dl_test_all = torch.utils.data.DataLoader(ds_test_all, batch_size=32, shuffle=False, num_workers=4)

In [9]:
# Train and validate the Baseline Classifier (without auxiliary loss)

model = Baseline()
trainer = pl.Trainer(
    max_epochs=10,
    progress_bar_refresh_rate=20
)
trainer.fit(model, dl_train_target, dl_test_target)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type             | Params
-------------------------------------------
0 | loss  | CrossEntropyLoss | 0     
1 | conv1 | Conv2d           | 608   
2 | conv2 | Conv2d           | 18.5 K
3 | flat  | Flatten          | 0     
4 | fc1   | Linear           | 25.7 K
5 | fc2   | Linear           | 202   
-------------------------------------------
45.0 K    Trainable params
0         Non-trainable params
45.0 K    Total params
0.180     Total estimated model params size (MB)
Epoch 0:  62%|██████▎   | 40/64 [00:01<00:01, 21.83it/s, loss=2.21, v_num=23, val_loss=5.030, val_acc=0.578]
Validating: 0it [00:00, ?it/s]
Epoch 1:  62%|██████▎   | 40/64 [00:02<00:01, 13.97it/s, loss=0.462, v_num=23, val_loss=0.575, val_acc=0.715]
Validating: 0it [00:00, ?it/s]
Epoch 2:  62%|██████▎   | 40/64 [00:04<00:02,  9.90it/s, loss=0.375, v_num=23, val_loss=0.600, val_acc=0.717]
Validating: 0it [00:00, ?it/s]
Epoch 3:  62%|████

1

In [12]:
# Train and validate the LeNet auxiliary network

model = LeNet()
trainer = pl.Trainer(
    max_epochs=10,
    progress_bar_refresh_rate=20
)
trainer.fit(model, dl_train_class, dl_test_class)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type             | Params
-------------------------------------------
0 | loss  | CrossEntropyLoss | 0     
1 | conv1 | Conv2d           | 320   
2 | conv2 | Conv2d           | 18.5 K
3 | flat  | Flatten          | 0     
4 | fc1   | Linear           | 51.4 K
5 | fc2   | Linear           | 2.0 K 
-------------------------------------------
72.2 K    Trainable params
0         Non-trainable params
72.2 K    Total params
0.289     Total estimated model params size (MB)
Epoch 0:  62%|██████▎   | 40/64 [00:03<00:01, 12.13it/s, loss=6.31, v_num=26, val_loss=15.70, val_acc=0.0938]
Validating: 0it [00:00, ?it/s]
Validating:   0%|          | 0/32 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fcc470c90d0>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 

1

In [3]:
# Train and validate the LeNet classifier (with auxiliary loss)
auxiliary = LeNet()
model = CombinedNet(auxiliary)
trainer = pl.Trainer(
    max_epochs=10,
    progress_bar_refresh_rate=20
)
trainer.fit(model, dl_train_all, dl_test_all)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name      | Type             | Params
-----------------------------------------------
0 | loss      | CrossEntropyLoss | 0     
1 | auxiliary | LeNet            | 72.2 K
2 | linear    | Linear           | 42    
-----------------------------------------------
72.3 K    Trainable params
0         Non-trainable params
72.3 K    Total params
0.289     Total estimated model params size (MB)
Epoch 0:  62%|██████▎   | 40/64 [00:02<00:01, 19.31it/s, loss=10.3, v_num=27, val_loss=1.920, val_acc=0.406]
Validating: 0it [00:00, ?it/s]
Epoch 1:  62%|██████▎   | 40/64 [00:02<00:01, 16.14it/s, loss=1.51, v_num=27, val_loss=0.621, val_acc=0.654]
Validating: 0it [00:00, ?it/s]
Epoch 2:  62%|██████▎   | 40/64 [00:02<00:01, 19.12it/s, loss=0.974, v_num=27, val_loss=0.527, val_acc=0.737]
Validating: 0it [00:00, ?it/s]
Epoch 3:  62%|██████▎   | 40/64 [00:02<00:01, 15.10it/s, loss=0.659, v_num=27, val_loss=0.459, val_acc=0.786]

1